In [ ]:
import pandas as pd
import helper_evaluation as helper_eval
import helper_data_load as helper_load
import random
import warnings
warnings.filterwarnings("ignore")
random_state= 42
random.seed(random_state)
import matplotlib
#matplotlib.style.use('bmh') 


In [ ]:
# Select feature selection method
feat_sel_method = "corr"
corr_meth = "spearman"

In [ ]:
if feat_sel_method == "mrmr":
    # if mrmr select to try how many solutions
    nr_sol_list = [1, 2, 3, 4]
    ##### Just read in the earlier build results
    # Lists for the different numbers of results list[0] = nr_sol1, list[1] = nr_sol2...
    discovery_chum_mrmr_results_list = []
    discovery_iucpq_mrmr_results_list = []
    for nr_sol in nr_sol_list:
        # Dicovery == CHUM, Validation == IUCPQ
        result_discovery_chum = pd.read_csv("../../Data/Results/os_days_results_CHUM_discovery_"+feat_sel_method+str(nr_sol)+"_std_scl_1-100.csv", index_col=0)
        discovery_chum_mrmr_results_list.append(result_discovery_chum)
        # Dicovery == IUCPQ, Validation == CHUM
        result_discovery_IUCPQ = pd.read_csv("../../Data/Results/os_days_results_IUCPQ_discovery_"+feat_sel_method+str(nr_sol)+"_std_scl_1-100.csv", index_col=0)
        discovery_iucpq_mrmr_results_list.append(result_discovery_IUCPQ)
elif feat_sel_method == "corr":
    result_discovery_chum_corr_list = []
    result_discovery_iucpq_corr_list = []
    if corr_meth == "spearman":
        nr_sol_list = [-1]
    elif corr_meth == "pearson":
        nr_sol_list = [-2]
    nr_sol = nr_sol_list[0]
    result_discovery_chum_corr_list.append(pd.read_csv("../../Data/Results/os_days_results_CHUM_discovery_"+feat_sel_method+str(nr_sol)+"_std_scl_1-100.csv", index_col=0))
    result_discovery_iucpq_corr_list.append(pd.read_csv("../../Data/Results/os_days_results_IUCPQ_discovery_"+feat_sel_method+str(nr_sol)+"_std_scl_1-100.csv", index_col=0))
elif feat_sel_method == "mim":
    result_discovery_chum_mim_list = []
    result_discovery_iucpq_mim_list = []
    nr_sol_list = [-1]
    nr_sol = nr_sol_list[0]
    result_discovery_chum_mim_list.append(pd.read_csv("../../Data/Results/os_days_results_CHUM_discovery_"+feat_sel_method+str(nr_sol)+"_std_scl_1-100.csv", index_col=0))
    result_discovery_iucpq_mim_list.append(pd.read_csv("../../Data/Results/os_days_results_IUCPQ_discovery_"+feat_sel_method+str(nr_sol)+"_std_scl_1-100.csv", index_col=0))

In [ ]:
def eval(discover_chum_results_list, discovery_iucpq_results_list, feat_sel_meth, corr_meth="pearson"):
    ### Plotting Discovery results
    save_plots = True
    discovery_chum_results_list_rename = [None] * len(nr_sol_list)
    discovery_iucpq_results_list_rename = [None] * len(nr_sol_list)
    discovery_chum_methods_nr_feat = [None] * len(nr_sol_list)
    discovery_iucpq_methods_nr_feat = [None] * len(nr_sol_list)
    dicovery_chum_meth_nr_feat_dict = [None] * len(nr_sol_list)
    dicovery_iucpq_meth_nr_feat_dict = [None] * len(nr_sol_list)
    df_vali_on_iucpq = [None] * len(nr_sol_list)
    df_vali_on_chum = [None] * len(nr_sol_list)
    df_vali_on_iucpq_rename = [None] * len(nr_sol_list)
    df_vali_on_chum_rename = [None] * len(nr_sol_list)

    data_norm_chum, data_norm_iucpq = helper_load.get_norm_data()
    for i, nr_sol in enumerate(nr_sol_list):
        #nr_sol = i+1
        #### Discovery == CHUM, Validation == IUCPQ ####
        # rename columns for plotting
        discovery_chum_results_list_rename[i] = helper_eval.rename_for_plotting(discover_chum_results_list[i])
        helper_eval.plot_result(discovery_chum_results_list_rename[i], "Discovery_CHUM "+feat_sel_method + str(nr_sol), save_plots, "_os_days_"+feat_sel_method+str(nr_sol)+"_std_scl_1-100")
        # Get the number of features corresponding to the best performance for each method
        discovery_chum_results_list_rename[i].set_index("Nr. Features", inplace=True)
        discovery_chum_methods_nr_feat[i] = helper_eval.get_nr_feat_per_method(discovery_chum_results_list_rename[i])
        discovery_chum_methods_nr_feat[i] = discovery_chum_methods_nr_feat[i].sort_values("C-Index", ascending=False, ignore_index=True)
        discovery_chum_methods_nr_feat[i].to_csv("../../Data/Results/CHUM_discovery_nr_feat"+"_"+feat_sel_method+str(nr_sol)+"_1-100.csv")
        # Build a dict which maps the method to the nr of features for best performance
        dicovery_chum_meth_nr_feat_dict[i] = helper_eval.get_meth_feat_dict(discovery_chum_methods_nr_feat[i], random_state=random_state)
        # Train all methods with best nr_feat on the whole discovery dataset and validate on validation set
        df_vali_on_iucpq[i] = helper_eval.train_disc_n_validate(data_norm_chum, data_norm_iucpq, dicovery_chum_meth_nr_feat_dict[i], feat_sel_meth, nr_sol, "IUCPQ")
        df_vali_on_iucpq[i].set_index("Method Name", inplace=True)
        # Save the validation results and plot them
        df_vali_on_iucpq[i].to_csv("../../Data/Results/os_days_results_best_model_validation_IUCPQ"+"_"+feat_sel_method+str(nr_sol)+"_1-100.csv")
        helper_eval.plot_vali_results(df_vali_on_iucpq[i], method="IUCPQ", feat_sel_method=feat_sel_method, nr_sol=nr_sol, random_state=random_state, save_plots=save_plots)
        # Prepare for general plot
        df_vali_on_iucpq_rename[i] = helper_eval.prep_method_name(df_vali_on_iucpq[i], random_state=random_state, name_to_func=False)
        discovery_chum_methods_nr_feat[i].rename(columns={"C-Index":"C-Index CHUM Discovery"}, inplace=True)
        
        
        #### Discovery == IUCPQ, Validation == CHUM ####
        # rename columns for plotting
        discovery_iucpq_results_list_rename[i] = helper_eval.rename_for_plotting(discovery_iucpq_results_list[i])
        helper_eval.plot_result(discovery_iucpq_results_list_rename[i], "Discovery_IUCPQ "+feat_sel_method + str(nr_sol), save_plots, "_os_days_"+feat_sel_method+str(nr_sol)+"_std_scl_1-100")
        # Get the number of features corresponding to the best performance for each method
        discovery_iucpq_results_list_rename[i].set_index("Nr. Features", inplace=True)
        discovery_iucpq_methods_nr_feat[i] = helper_eval.get_nr_feat_per_method(discovery_iucpq_results_list_rename[i])
        discovery_iucpq_methods_nr_feat[i] = discovery_iucpq_methods_nr_feat[i].sort_values("C-Index", ascending=False, ignore_index=True)
        discovery_iucpq_methods_nr_feat[i].to_csv("../../Data/Results/IUCPQ_discovery_nr_feat"+"_"+feat_sel_method+str(nr_sol)+"_1-100.csv")
        # Build a dict which maps the method to the nr of features for best performance
        dicovery_iucpq_meth_nr_feat_dict[i] = helper_eval.get_meth_feat_dict(discovery_iucpq_methods_nr_feat[i], random_state=random_state)
        # Train all methods with best nr_feat on the whole discovery dataset and validate on validation set
        df_vali_on_chum[i] = helper_eval.train_disc_n_validate(data_norm_iucpq, data_norm_chum, dicovery_iucpq_meth_nr_feat_dict[i], feat_sel_meth, nr_sol, "CHUM")
        df_vali_on_chum[i].set_index("Method Name", inplace=True)
        # Save the validation results and plot them
        df_vali_on_chum[i].to_csv("../../Data/Results/os_days_results_best_model_validation_CHUM"+"_"+feat_sel_method+str(nr_sol)+"_1-100.csv")
        helper_eval.plot_vali_results(df_vali_on_chum[i], method="CHUM", feat_sel_method=feat_sel_method, nr_sol=nr_sol, random_state=random_state, save_plots=save_plots)
        # Plot all results in one
        df_vali_on_chum_rename[i] = helper_eval.prep_method_name(df_vali_on_chum[i], random_state=random_state, name_to_func=False)
        discovery_iucpq_methods_nr_feat[i].rename(columns={"C-Index":"C-Index IUCPQ Discovery"}, inplace=True)

        helper_eval.plot_vali_results_in_one(df_vali_on_chum_rename[i], df_vali_on_iucpq_rename[i], discovery_chum_methods_nr_feat[i], discovery_iucpq_methods_nr_feat[i], feat_sel_method, nr_sol, save_plots=save_plots)

        if feat_sel_method == "corr":
            break


In [ ]:
if feat_sel_method == "corr":
    eval(result_discovery_chum_corr_list, result_discovery_iucpq_corr_list, feat_sel_method)
elif feat_sel_method == "mrmr":
    eval(discovery_chum_mrmr_results_list, discovery_iucpq_mrmr_results_list, feat_sel_method)
elif feat_sel_method == "mim":
    eval(result_discovery_chum_mim_list, result_discovery_iucpq_mim_list, feat_sel_method)